## DBD Plasma Physics correlation with Python Code

### [용어 정리]

### Mobility

Electrical mobility is **the ability of charged particles (such as electrons or protons) to move** through a medium in **response to an electric field** that is pulling them.
<br>- 모빌리티는 물질 고유의 특성값이고 에너지(온도)와의 상관관계가 존재한다. 

<img src=".\06_fig\10.gif" "width=300" />

#### A. Mobility 관계식

- Ion Mobility : function of E/N
- Electron Mobility : function of $\bar \varepsilon$

#### B. Why?

Ion Mobility

- energy distribution을 electric field 값으로 근사할 수 있다. (local field approximation)
- E/N으로 나타내는 것이 일반적이다. (E/N : ratio of the electric field strength to the gas particle number density)
    - where the ratio of E/N is small and thus the thermal energy of the ions is much greater than the energy gained from the electric field between collisions.
- https://fr.lxcat.net 에서 E/N 대비 Mobility Data를 구할 수 있다.

Electron Mobility

- local field approximation을 활용할 수 없다. (에너지와의 상관관계를 직접 활용)
- BOLSIG+ 를 활용하여 $\bar \varepsilon$ 대비 Mobility Data을 찾아낼 수 있다.

#### C. Data 예시

Ion Mobility
<img src=".\06_fig\00_ion_mobility.png" "width=400" />

Electron Mobility
<img src=".\06_fig\01_BOLSIG.png" "width=400" />

#### D. 결론

이온 이동도
1. E/N의 함수이다.
2. https://fr.lxcat.net 에서 Mobility Data를 구할 수 있다.

전자 이동도
1. $\bar \varepsilon$의 함수이다.
2. BOLSIG+를 활용하여 Mobility Data를 구할 수 있다.

<br>
## 1. Physics

### (1) 모식도

<img src=".\06_fig\02.png" "width=300" />

<img src=".\06_fig\03.png" "width=300" />

<img src=".\06_fig\04.png" "width=300" />

<img src=".\06_fig\05.png" "width=300" />

<img src=".\06_fig\06.png" "width=300" />

<img src=".\06_fig\07.png" "width=260" />

### (2) 핵심 현상

다음의 세 가지 현상을 모델링 하는 것이 핵심이다.

<img src=".\06_fig\07-1.png" "width=900" />

1. Continuity Equation
    - 입자 이동 및 이온화 계산 : 밀도 분포 변화
2. Poisson's Equation
    - 전기장 세기 계산
      - E/N : Ion Mobility 값
      - 연속방정식에서의 convective flux에 영향
3. Energy Equation
    - 전자 에너지 계산
      - $\bar\varepsilon$ : Electron Mobility 값
4. Equation of charge accumulation on surface of dielectric
    - 표면에 축적되는 양이온 입자 밀도 계산

### (3) 유의사항

1. 교류 전압
    - 교류 전압이므로, 시간의 흐름에 따라 전위 경계조건을 바꿔주어야 한다.
2. 이온 밀도
    - 양이온이 표면에 축적되므로, 시간의 흐름에 따라 연속방정식 경계조건을 바꿔주어야 한다.

<br>
## 2. Modeling : 1D

<img src=".\06_fig\08_Modeling.png" "width=500" />

### (1) 핵심 파라미터 : $\mu, D, k$
-  $\mu, D, k$는 위치의 함수
  - 위치에 따라 $E/N$ 값이 다르다.
  - 위치에 따라 $\bar\varepsilon$ 값이 다르다.


- 시간 흐름에 따른 재설정
  - 시간에 따라 $E/N, \bar\varepsilon$ 값이 변한다.
  - Loop System 도입

### (2) 알고리즘

<img src=".\06_fig\09.png" "width=800" />

<br>
## 3. Python Code

이 자료에서는 크게 유의해야 할 내용 위주로 점검한다. 세부적인 내용은 추후 다룰 예정.

### (1) Initialization

- Grid(위치) 별로 계산함을 확인할 수 있다.

In [2]:
import numpy as np
import sys
import matplotlib.pyplot as plt
import scipy.sparse.linalg as la
import scipy.sparse as sparse
import time as tm

#-----------------parameters for the plasma reactor---------------------------

ee=1.6*10**(-19)              #electronic charge
width=2.0                     #space between two dielectric in mm
ngrid0=3                      #Number of grid points (between two dielectric)
wd1=1.5                       #width of first dielectric in mm
wd2=1.5                       #width of second dielectric in mm
dx=width*10**(-3)/(ngrid0+1.0)#Grid size in meter
nwd1=int(wd1*10**(-3)/dx)     #number of grid points in first dielectric
nwd2=int(wd2*10**(-3)/dx)     #Number of grid points in second dielectric
wd1=nwd1*dx                   #Making wd1 as exact multiple of dx
wd2=nwd2*dx                   #making wd2 as exact multiple of dx
inelec=width*10**(-3)+wd1+wd2 #total interelectrode separation
ngrid=int(ngrid0+2+nwd1+nwd2) #total number of grid points(2 dielectrics +gas medium + all edge points)

#*** Initialization
#-----------------------------------------------------------------------------------------------------

ns=4                                    #total number of species
nr=5                                    #total number of chemical reactions
ndensity=np.zeros((ns,ngrid0+2),float)  #number density of each species
ncharge=np.array([-1,0,0,1])            #corresponding charge of the each species
gMat=np.array([0,0,0,1])                #gamma matrix (boolean what produces secondary electrons)
dMat=np.array([1,0,0,0])                #boolean, which species undergoes desportion from surface
netcharge=np.zeros(ngrid,float)         #net charge at each grid points
potentl=np.zeros(ngrid,float)           #potential at each grid points
efield=np.zeros(ngrid0+2,float)         #electric field at each grid points
efieldP=np.zeros(ngrid0+2,float)        #electric field at each grid points

mobilityG=np.zeros((ns,ngrid0+2),float) #mobility at each grid points
diffusionG=np.zeros((ns,ngrid0+2),float)#diffusion coefficient at grid points
sourceG=np.zeros((nr,ngrid0+2),float)   #source at each grid points
fluxLR=np.zeros((ns,2),float)           #particle flux towards left and right boundries
react=np.zeros((4,ngrid0+2),float)      #reaction 
sigmaLR=np.zeros((ns,2),float)          #surface charge density
tempSigma=np.zeros((ns,2),float)        #temporary surface charge density
eEjection=np.zeros((2),float)           #electrons leaving the boundary

<br>
<img src=".\06_fig\08_Modeling.png" "width=500" />

In [3]:
ndensity.shape, mobilityG.shape, diffusionG.shape

((4, 5), (4, 5), (4, 5))

위의 세 가지 Data는 입자의 움직임과 연관되어 있다. 따라서 유전체 사이 Grid 영역만 고려한다.
- 4행 : 입자 species (전자, 중성, exciting, ionization)
- 5열 : 유전체 사이 Grid (5개)

<br>
### (2) Posson's Eq'n

$$\nabla\cdot(\varepsilon \mathbf{E})=-\nabla\cdot( \varepsilon \nabla V)=\rho$$
$$ \rho : \rho(x) $$

- Poisson's Eq'n의 우항인 Netcharge는 Grid 영역에 따라 다름을 고려한다.

In [4]:
#--------------------------initial netcharge-----------------------------------
ndensity=1000*np.random.rand(ns,ngrid0+2)
ncharge=np.array([-1,1,0,0])     # 전자, ion, exciting, 중성

netcharge=np.zeros(ngrid,float)
netcharge[nwd1:nwd1+2+ngrid0]=ee*np.dot(ncharge,ndensity)   

<br>
<img src=".\06_fig\08_Modeling.png" "width=500" />

In [5]:
netcharge

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  9.51393281e-17,
        5.05872383e-17, -1.18356160e-16,  6.10845358e-18,  1.14572621e-16,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

<br>
### (3) Parameters

- $\mu, D, k$ 값의 변화를 고려한다.

In [6]:
####POISSON MATRIX USED TO SOLVE THE POISSON'S EQUATION IMPLICTLY====
def SparseLaplacianOperator(nx,k1=-1,k2=0,k3=1):
    d1=np.zeros((nx),float)
    d2=np.ones((nx),float)
    d3=np.zeros((nx),float)
    d1[:-2]=1
    d2[1:-1]=-2
    d3[2:]=1
    return (sparse.dia_matrix(([d1,d2,d3],[k1,k2,k3]),shape=(nx,nx)).tocsc() )

####IMPORTING THE TRANSPORT AND REACTION COEFFICIENTS FROM TEXT FILE======================================
def readBoltzmannParameters1(npoints,oupfile='table.txt',ns=4,nR=5):
    mobility=np.zeros((ns,npoints),float)
    diffusion=np.zeros((ns,npoints),float)
    source=np.zeros((nR,npoints),float)
    file=open(oupfile)
    line=file.readline()
    for data in np.arange(npoints):
       line=file.readline()
       lineSplit=line.split()
       mobility[0,data]=float(lineSplit[0]);mobility[1,data]=float(lineSplit[1]);mobility[2,data]=float(lineSplit[2])
       diffusion[0,data]=float(lineSplit[3]);diffusion[1,data]=float(lineSplit[4]);diffusion[2,data]=float(lineSplit[5])
       diffusion[3,data]=float(lineSplit[6])
       source[0,data]=float(lineSplit[7]);source[1,data]=float(lineSplit[8]);source[2,data]=float(lineSplit[9])
       source[3,data]=float(lineSplit[10]);source[4,data]=float(lineSplit[11])
    return(mobility,diffusion,source)

#SPARSE TRIDIAGONAL MATRIX USED TO SOLVE THE DIFFUSION EQUATION IMPLICTLY====================================
def SparseDiffusionOperator(numberdensity,dif,dx,dt,k1=-1,k2=0,k3=1):
    nx=dif.size
    d1=np.zeros((nx),float)
    d2=np.ones((nx),float)
    d3=np.zeros((nx),float)
    d1[:-2]=(dt/(4*dx*dx))*(dif[2:]-dif[:-2]-4*dif[1:-1])
    d2[1:-1]=(1+2*dt*dif[1:-1]/(dx**2))
    d3[2:]=(dt/(4*dx*dx))*(-dif[2:]+dif[:-2]-4*dif[1:-1])
    return (la.spsolve((sparse.dia_matrix(([d1,d2,d3],[k1,k2,k3]),shape=(nx,nx)).tocsc()),numberdensity))

#INTERPOLATION FORMULA ======================================================================================
def Interpolation(efield,inputdata):
    indlocate=abs(efield[:]).astype(int)
    return(inputdata[:,indlocate]+(inputdata[:,indlocate+1]-inputdata[:,indlocate])*(abs(efield)-indlocate))

#ADVECTION SOLVING MATRIX====================================================================================
def AdvectionAlgorithm(dx,dt,velocity,density):
    flux=(0.5*(velocity[1:]*density[1:]+velocity[:-1]*density[:-1])-0.5*0.5*abs(velocity[1:]+velocity[:-1])*(density[1:]-density[:-1]))*dt
    density[1:-1]+= -(flux[1:]-flux[:-1])/dx
    return density[1:-1]

start = tm.time()

parameterSize=996 #NUMBER OF ROWS IN THE INPUT TEXT FILE
(mobilityInput,diffusionInput,sourceInput) = readBoltzmannParameters1(parameterSize,'table.txt')# reading from the input file
mobilityInput[1]=mobilityInput[1]/0.12*1.6   #correction
diffusionInput[1]=diffusionInput[1]/0.12*1.6 #correction
mobilityInput[2]=mobilityInput[2]/0.12*1.6   #correction
diffusionInput[2]=diffusionInput[2]/0.12*1.6 #correction

In [7]:
poissonSparseMatrix=SparseLaplacianOperator(ngrid)  #poisson equation solving matrix
numberOfSteps = 1000                                 #total number of time steps
stepping=2                                          #time steps to skip before saving the data

storedensity=np.zeros((int(numberOfSteps/stepping),ns,ngrid0+2),float)          #number density    
storenetcharge=np.zeros((int(numberOfSteps/stepping),ngrid0+2+nwd1+nwd2),float) #net charge
storeefield=np.zeros((int(numberOfSteps/stepping),ngrid0+2),float)              #elecritc field
storepotentl=np.zeros((int(numberOfSteps/stepping),ngrid0+2+nwd1+nwd2),float)   #potential
#storeReact=np.zeros((int(numberOfSteps/stepping),ns,ngrid0+2),float)           #production rate
#storeR=np.zeros((int(numberOfSteps/stepping),nr,ngrid0+2),float)               #reaction rate
storeCurrent=np.zeros((int(numberOfSteps/stepping)),float)                      #current
ndensity=1000*np.random.rand(ns,ngrid0+2) #initializing the number densities with random value 

In [8]:
volt=5000.0                   #Interelectrode voltage (peak not RMS)
gasdens=2.504e25              #number density of gas at NTP (unit: m^-3)
dt=1.0e-10                    #small time interval
frequencySource =  41000       #in Hz
ee=1.6*10**(-19)              #electronic charge
e0=8.54187817*10**(-12)       #epsilon
townsendunit=1.0/((gasdens)*(10)**(-21))#townsend factor
Kboltz=1.380*10e-23           #Boltzmann constant
desorption=1*10.                #desorption coefficient
recombination=1e-10           #surface recombination coefficient
gamma=0.01                    #secondary electron emission coefficient
#======================================================================================================
time=0.0
#=======================time Loop======================================================================
for tymeStep in range(1,numberOfSteps):
    time=time+dt

    
    #                           *** POISSON'S EQUATION ***
    #-------------------------------------------------------------------------------------------
    netcharge[nwd1:nwd1+2+ngrid0]=ee*np.dot(ncharge,ndensity)   #calculating net charge
    netcharge[nwd1+1:nwd1+1+ngrid0]=0.                          #quasi neutrality condition
    leftPot=1.0*volt*np.sin(2*np.pi*time*frequencySource)       #applied voltage (left)
    rightpot=0.0*volt*np.sin(2*np.pi*time*frequencySource)      #applied voltage (right)
    chrgg=-(netcharge/e0)*dx*dx                                 #RHS matrix. <Read documentation>
    chrgg[0]=leftPot                                            #left boundary condition
    chrgg[-1]=rightpot                                          #right boundary condition
    potentl=la.spsolve(poissonSparseMatrix,chrgg)               #solving system of Matrix equations
    
    #**calculate electric field as negative gradient of potential (Expressed in Townsend Unit)
    efield[:]=-townsendunit*(potentl[nwd1+1:nwd1+3+ngrid0]-potentl[nwd1-1:nwd1+1+ngrid0])/(2.0*dx)
    efield[0]=-townsendunit*(-(11.0/6)*potentl[nwd1]+3.0*potentl[nwd1+1]-(3.0/2)*potentl[nwd1+2]+(1.0/3)*potentl[nwd1+3])/dx
    efield[-1]=-townsendunit*(potentl[nwd1+1+ngrid0]-potentl[nwd1+ngrid0])/dx
    
    if any(abs(efield[:])>995):#Stop the program if E>995 Townsends.
         sys.exit()
    
    
    #                      *** TRANSPORT AND REACTION COEFFICIENTS ***
    #----------------------------------------------------------------------------------------------
    mobilityG=np.transpose(ncharge*np.transpose(Interpolation(efield,mobilityInput))) #mobility
    diffusionG=Interpolation(efield,diffusionInput)                                   #diffusion
    sourceG=Interpolation(efield,sourceInput)                                         #reaction rate
    efield[:]=efield[:]/townsendunit #converting Efield back to SI(V/m) unit from Townsend's unit

    
    #                               *** SOURCE TERM ***
    #-----------------------------------------------------------------------------------------------
    R1=sourceG[0]*(ndensity[0]+1*10000)*gasdens*dt        #source of reaction-1 <read documentation>
    R2=1*sourceG[1]*(ndensity[0]+1*10000)*ndensity[3]*dt  #source of reaction-2 <read documentation>
    R3=1*gasdens*gasdens*sourceG[2]*ndensity[1]*dt        #source of reaction-3 <read documentation>
    R4=1*sourceG[3]*(ndensity[0]+1*10000)*ndensity[2]*dt  #source of reaction-4 <read documentation>
    R5=1*(ndensity[0]+1*10000)*gasdens*sourceG[4]*dt      #source of reaction-5 <read documentation>
    
    react[0]=(R1+R2-R4)                 #production of particle [0]
    react[1]=(R1+R2-R3)                 #production of particle [1]
    react[2]=(R3-R4)                    #production of particle [2]
    react[3]=(-R2+R5)                   #production of particle [3]
    ndensity[:,1:-1]+= react[:,1:-1]    #adding newly produced particles to the gas
    
    
    #                            *** CURRENT CALCULATION ***
    #------------------------------------------------------------------------------------------------
    current=ee*np.sum((efield[2:-2]*mobilityG[1,2:-2]*ndensity[1,2:-2]
                                                                  +  1*efield[2:-2]*mobilityG[2,2:-2]*ndensity[2,2:-2]+   
                                                                  efield[2:-2]*mobilityG[0,2:-2]*ndensity[0,2:-2]-  
                                                                  diffusionG[2,2:-2]*(ndensity[2,3:-1]  -ndensity[2,1:-3])/(2*dx)
                                                                  -diffusionG[1,2:-2]*(ndensity[1,3:-1]-ndensity[1,1:-3])/(2*dx)
                                                                  +diffusionG[0,2:-2]*(ndensity[0,3:-1]-ndensity[0,1:-3])/(2*dx))*dx)

    
    #                                    *** DIFFUSION ***
    #-------------------------------------------------------------------------------------------------------
    temporaryCopy=ndensity.copy()                                                 #making a copy of nDensity 
    temporaryCopy[:,0]=temporaryCopy[:,1].copy()                                  #mirror boundary (left)
    temporaryCopy[:,-1]=temporaryCopy[:,-2].copy()                                #mirror boundary (right)
    temporaryCopy[0]=SparseDiffusionOperator(temporaryCopy[0],diffusionG[0],dx,dt)#solving Implictly for[0]
    temporaryCopy[1]=SparseDiffusionOperator(temporaryCopy[1],diffusionG[1],dx,dt)#solving Implictly for[1]
    temporaryCopy[2]=SparseDiffusionOperator(temporaryCopy[2],diffusionG[2],dx,dt)#solving Implictly for[2]
    temporaryCopy[3]=SparseDiffusionOperator(temporaryCopy[3],diffusionG[3],dx,dt)#solving Implictly for[3]
    
    
    #                                   *** ADVECTION ***
    #-------------------------------------------------------------------------------------------------------
    temporaryCopy[:-1,0]=fluxLR[:-1,0]*dt/dx              #left boundary condition
    temporaryCopy[:-1,-1]=fluxLR[:-1,1]*dt/dx             #right boundary condition
        
    temporaryCopy[0,1:-1]=AdvectionAlgorithm(dx,dt,mobilityG[0]*efield,temporaryCopy[0]) #solving for [0]
    temporaryCopy[1,1:-1]=AdvectionAlgorithm(dx,dt,mobilityG[1]*efield,temporaryCopy[1]) #solving for [1]
    temporaryCopy[2,1:-1]=AdvectionAlgorithm(dx,dt,mobilityG[2]*efield,temporaryCopy[2]) #solving for [2]
    ndensity[:,1:-1]=temporaryCopy[:,1:-1].copy()#copying back
    
    
    #                           ***BOUNDARY *** (charge accumulation at surface of dielectric)
    #--------------------------------------------------------------------------------------------------------
    recoMat=recombination*np.array([[  sigmaLR[0,0]*sigmaLR[1,0]+sigmaLR[0,0]*sigmaLR[2,0],sigmaLR[0,0]*sigmaLR[1,0],sigmaLR[0,0]*sigmaLR[2,0],0.],
                      [sigmaLR[0,1]*sigmaLR[1,1]+sigmaLR[0,1]*sigmaLR[2,1],sigmaLR[0,1]*sigmaLR[1,1],sigmaLR[0,1]*sigmaLR[2,1],0.]])
                      #recoMat is the MATRIX used for calculating electron-ion recombination on dielectric surfaces
    velocity=mobilityG*efield                                                             #velocity of particles
    fluxLR[:,0]=-(ndensity[:,1]*velocity[:,1]+gamma*gMat*ndensity[:,1]*velocity[:,1])     #flux at left dielectric
    fluxLR[:,1]=(ndensity[:,-2]*velocity[:,-2]+gamma*gMat*ndensity[:,-2]*velocity[:,-2])  #flux at right dielectric
    fluxLR[fluxLR<0]=0.                                                                   #non negative condition 
    sigmaLR[:,0]+=dt*(fluxLR[:,0]-desorption*dMat*sigmaLR[:,0]-recoMat[0])                #surface charge density (left dielectric)
    sigmaLR[:,1]+=dt*(fluxLR[:,1]-desorption*dMat*sigmaLR[:,1]-recoMat[1])                #surface charge density(right dielectric)
    #------
    secondary1=-gamma*(ndensity[1,1]*velocity[1,1]+ndensity[2,1]*velocity[2,1])           #secondary electron emission on left dielectric
    secondary2=gamma*(ndensity[1,-2]*velocity[1,-2]+ndensity[2,-2]*velocity[2,-2])        #secondary electron emission on right dielectric
    if secondary1<0:secondary1=0                                                          #non-negative condition
    if secondary2<0:secondary2=0                                                          #non-negative condition
    ndensity[0,1:5]=(dt*(desorption*sigmaLR[0,0]/4+secondary1/4)+ndensity[0,1:5]*dx)/dx   #electron added near left boundary (desorption+secondary emission)
    ndensity[0,-6:-2]=(dt*(desorption*sigmaLR[0,1]/4+secondary2/4)+ndensity[0,-6:-2]*dx)/dx#electron added near right boundary (desorption+secondary emission)
    #------             
    ndensity[:-1,0]=sigmaLR[:-1,0]/dx    #volume charge density approximation due to charge accumulation on left dielectric            
    ndensity[:-1,-1]=sigmaLR[:-1,1]/dx   #volume charge density approximation due to charge accumulation on left dielectric
    ndensity[:,ndensity[0]<0]=0.         #non-negative number of particles
    
    
    #                   *** DATA STORAGE ***
    #---------------------------------------------------------
    if (tymeStep%stepping==0):
        #print(mobilityG[0,10],mobilityG[1,10],mobilityG[2,10])
        storedensity[int(tymeStep/stepping),:,:]=ndensity[:,:]
        storenetcharge[int(tymeStep/stepping)]=netcharge
        storeefield[int(tymeStep/stepping)]=efield
        storepotentl[int(tymeStep/stepping)]=potentl
        storeCurrent[int(tymeStep/stepping)]=current

In [9]:
mobilityG

array([[-0.11885699, -0.11885699, -0.11885699, -0.11885699, -0.11885699],
       [ 0.01624682,  0.01624682,  0.01624682,  0.01624682,  0.01624682],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [10]:
numberOfSteps = 10

time=0.0
#=======================time Loop======================================================================
for tymeStep in range(1,numberOfSteps):
    time=time+dt

    
    #                           *** POISSON'S EQUATION ***
    #-------------------------------------------------------------------------------------------
    netcharge[nwd1:nwd1+2+ngrid0]=ee*np.dot(ncharge,ndensity)   #calculating net charge
    netcharge[nwd1+1:nwd1+1+ngrid0]=0.                          #quasi neutrality condition
    leftPot=1.0*volt*np.sin(2*np.pi*time*frequencySource)       #applied voltage (left)
    rightpot=0.0*volt*np.sin(2*np.pi*time*frequencySource)      #applied voltage (right)
    chrgg=-(netcharge/e0)*dx*dx                                 #RHS matrix. <Read documentation>
    chrgg[0]=leftPot                                            #left boundary condition
    chrgg[-1]=rightpot                                          #right boundary condition
    potentl=la.spsolve(poissonSparseMatrix,chrgg)               #solving system of Matrix equations
    
    #**calculate electric field as negative gradient of potential (Expressed in Townsend Unit)
    efield[:]=-townsendunit*(potentl[nwd1+1:nwd1+3+ngrid0]-potentl[nwd1-1:nwd1+1+ngrid0])/(2.0*dx)
    efield[0]=-townsendunit*(-(11.0/6)*potentl[nwd1]+3.0*potentl[nwd1+1]-(3.0/2)*potentl[nwd1+2]+(1.0/3)*potentl[nwd1+3])/dx
    efield[-1]=-townsendunit*(potentl[nwd1+1+ngrid0]-potentl[nwd1+ngrid0])/dx
    
    if any(abs(efield[:])>995):#Stop the program if E>995 Townsends.
         sys.exit()
    
    
    #                      *** TRANSPORT AND REACTION COEFFICIENTS ***
    #----------------------------------------------------------------------------------------------
    mobilityG=np.transpose(ncharge*np.transpose(Interpolation(efield,mobilityInput))) #mobility
    diffusionG=Interpolation(efield,diffusionInput)                                   #diffusion
    sourceG=Interpolation(efield,sourceInput)                                         #reaction rate
    efield[:]=efield[:]/townsendunit #converting Efield back to SI(V/m) unit from Townsend's unit

    
    #                               *** SOURCE TERM ***
    #-----------------------------------------------------------------------------------------------
    R1=sourceG[0]*(ndensity[0]+1*10000)*gasdens*dt        #source of reaction-1 <read documentation>
    R2=1*sourceG[1]*(ndensity[0]+1*10000)*ndensity[3]*dt  #source of reaction-2 <read documentation>
    R3=1*gasdens*gasdens*sourceG[2]*ndensity[1]*dt        #source of reaction-3 <read documentation>
    R4=1*sourceG[3]*(ndensity[0]+1*10000)*ndensity[2]*dt  #source of reaction-4 <read documentation>
    R5=1*(ndensity[0]+1*10000)*gasdens*sourceG[4]*dt      #source of reaction-5 <read documentation>
    
    react[0]=(R1+R2-R4)                 #production of particle [0]
    react[1]=(R1+R2-R3)                 #production of particle [1]
    react[2]=(R3-R4)                    #production of particle [2]
    react[3]=(-R2+R5)                   #production of particle [3]
    ndensity[:,1:-1]+= react[:,1:-1]    #adding newly produced particles to the gas
    
    
    #                            *** CURRENT CALCULATION ***
    #------------------------------------------------------------------------------------------------
    current=ee*np.sum((efield[2:-2]*mobilityG[1,2:-2]*ndensity[1,2:-2]
                                                                  +  1*efield[2:-2]*mobilityG[2,2:-2]*ndensity[2,2:-2]+   
                                                                  efield[2:-2]*mobilityG[0,2:-2]*ndensity[0,2:-2]-  
                                                                  diffusionG[2,2:-2]*(ndensity[2,3:-1]  -ndensity[2,1:-3])/(2*dx)
                                                                  -diffusionG[1,2:-2]*(ndensity[1,3:-1]-ndensity[1,1:-3])/(2*dx)
                                                                  +diffusionG[0,2:-2]*(ndensity[0,3:-1]-ndensity[0,1:-3])/(2*dx))*dx)

    
    #                                    *** DIFFUSION ***
    #-------------------------------------------------------------------------------------------------------
    temporaryCopy=ndensity.copy()                                                 #making a copy of nDensity 
    temporaryCopy[:,0]=temporaryCopy[:,1].copy()                                  #mirror boundary (left)
    temporaryCopy[:,-1]=temporaryCopy[:,-2].copy()                                #mirror boundary (right)
    temporaryCopy[0]=SparseDiffusionOperator(temporaryCopy[0],diffusionG[0],dx,dt)#solving Implictly for[0]
    temporaryCopy[1]=SparseDiffusionOperator(temporaryCopy[1],diffusionG[1],dx,dt)#solving Implictly for[1]
    temporaryCopy[2]=SparseDiffusionOperator(temporaryCopy[2],diffusionG[2],dx,dt)#solving Implictly for[2]
    temporaryCopy[3]=SparseDiffusionOperator(temporaryCopy[3],diffusionG[3],dx,dt)#solving Implictly for[3]
    
    
    #                                   *** ADVECTION ***
    #-------------------------------------------------------------------------------------------------------
    temporaryCopy[:-1,0]=fluxLR[:-1,0]*dt/dx              #left boundary condition
    temporaryCopy[:-1,-1]=fluxLR[:-1,1]*dt/dx             #right boundary condition
        
    temporaryCopy[0,1:-1]=AdvectionAlgorithm(dx,dt,mobilityG[0]*efield,temporaryCopy[0]) #solving for [0]
    temporaryCopy[1,1:-1]=AdvectionAlgorithm(dx,dt,mobilityG[1]*efield,temporaryCopy[1]) #solving for [1]
    temporaryCopy[2,1:-1]=AdvectionAlgorithm(dx,dt,mobilityG[2]*efield,temporaryCopy[2]) #solving for [2]
    ndensity[:,1:-1]=temporaryCopy[:,1:-1].copy()#copying back
    
    
    #                           ***BOUNDARY *** (charge accumulation at surface of dielectric)
    #--------------------------------------------------------------------------------------------------------
    recoMat=recombination*np.array([[  sigmaLR[0,0]*sigmaLR[1,0]+sigmaLR[0,0]*sigmaLR[2,0],sigmaLR[0,0]*sigmaLR[1,0],sigmaLR[0,0]*sigmaLR[2,0],0.],
                      [sigmaLR[0,1]*sigmaLR[1,1]+sigmaLR[0,1]*sigmaLR[2,1],sigmaLR[0,1]*sigmaLR[1,1],sigmaLR[0,1]*sigmaLR[2,1],0.]])
                      #recoMat is the MATRIX used for calculating electron-ion recombination on dielectric surfaces
    velocity=mobilityG*efield                                                             #velocity of particles
    fluxLR[:,0]=-(ndensity[:,1]*velocity[:,1]+gamma*gMat*ndensity[:,1]*velocity[:,1])     #flux at left dielectric
    fluxLR[:,1]=(ndensity[:,-2]*velocity[:,-2]+gamma*gMat*ndensity[:,-2]*velocity[:,-2])  #flux at right dielectric
    fluxLR[fluxLR<0]=0.                                                                   #non negative condition 
    sigmaLR[:,0]+=dt*(fluxLR[:,0]-desorption*dMat*sigmaLR[:,0]-recoMat[0])                #surface charge density (left dielectric)
    sigmaLR[:,1]+=dt*(fluxLR[:,1]-desorption*dMat*sigmaLR[:,1]-recoMat[1])                #surface charge density(right dielectric)
    #------
    secondary1=-gamma*(ndensity[1,1]*velocity[1,1]+ndensity[2,1]*velocity[2,1])           #secondary electron emission on left dielectric
    secondary2=gamma*(ndensity[1,-2]*velocity[1,-2]+ndensity[2,-2]*velocity[2,-2])        #secondary electron emission on right dielectric
    if secondary1<0:secondary1=0                                                          #non-negative condition
    if secondary2<0:secondary2=0                                                          #non-negative condition
    ndensity[0,1:5]=(dt*(desorption*sigmaLR[0,0]/4+secondary1/4)+ndensity[0,1:5]*dx)/dx   #electron added near left boundary (desorption+secondary emission)
    ndensity[0,-6:-2]=(dt*(desorption*sigmaLR[0,1]/4+secondary2/4)+ndensity[0,-6:-2]*dx)/dx#electron added near right boundary (desorption+secondary emission)
    #------             
    ndensity[:-1,0]=sigmaLR[:-1,0]/dx    #volume charge density approximation due to charge accumulation on left dielectric            
    ndensity[:-1,-1]=sigmaLR[:-1,1]/dx   #volume charge density approximation due to charge accumulation on left dielectric
    ndensity[:,ndensity[0]<0]=0.         #non-negative number of particles
    
    
    #                   *** DATA STORAGE ***
    #---------------------------------------------------------
    if (tymeStep%stepping==0):
        #print(mobilityG[0,10],mobilityG[1,10],mobilityG[2,10])
        storedensity[int(tymeStep/stepping),:,:]=ndensity[:,:]
        storenetcharge[int(tymeStep/stepping)]=netcharge
        storeefield[int(tymeStep/stepping)]=efield
        storepotentl[int(tymeStep/stepping)]=potentl
        storeCurrent[int(tymeStep/stepping)]=current

In [11]:
mobilityG

array([[-0.12020767, -0.12020767, -0.12020767, -0.12020767, -0.12020767],
       [ 0.01651766,  0.01651766,  0.01651766,  0.01651766,  0.01651766],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

<br>
### (4) 표면 전하 축적

In [12]:
ndensity

array([[1.85372325e+02, 5.67657797e+02, 4.40607817e+02, 3.48011102e+02,
        0.00000000e+00],
       [0.00000000e+00, 1.05603570e-04, 4.37891010e-05, 8.73064471e-05,
        2.50340014e-01],
       [0.00000000e+00, 1.28823228e+03, 6.74559967e+02, 1.49079619e+03,
        0.00000000e+00],
       [1.16806976e+02, 1.03281855e+04, 1.10228359e+04, 1.09015357e+04,
        6.77389259e+02]])